In [1]:
import os
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import warnings
warnings.filterwarnings('ignore')


/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [27]:
data = pd.read_csv('/Users/caseyh/Desktop/footyfootyfooty/data_wrangling/round_data.csv')
data['target'] = data['Home Score'] - data['Away Score']


In [28]:
data['Venue'].replace(' ','_',regex=True,inplace=True)

In [29]:
df = data.loc[(data['round']!=1)&~((data['year']==2023)&(data['round']>17))]

In [30]:
current = data.loc[((data['year']==2023)&(data['round']==18))]
current

,Date,Venue,Crowd,round,year,Home,Away,Home Score,Away Score,target,...,s_round_away,position_away,result_home_l5,pf_home_l5,pa_home_l5,game played_home_l5,result_away_l5,pf_away_l5,pa_away_l5,game played_away_l5
4204,Thu 13 Jul 7:20pm,SCG,0.0,18,2023,SYD,WBD,0.0,0.0,0.0,...,18.0,7.0,6.0,481.0,353.0,5.0,8.0,369.0,353.0,4.0
4205,Fri 14 Jul 7:50pm,MCG,0.0,18,2023,MEL,BRL,0.0,0.0,0.0,...,18.0,3.0,8.0,253.0,245.0,4.0,16.0,504.0,323.0,5.0
4206,Sat 15 Jul 1:45pm,MCG,0.0,18,2023,COL,FRE,0.0,0.0,0.0,...,18.0,14.0,12.0,353.0,265.0,4.0,4.0,317.0,452.0,5.0
4207,Sat 15 Jul 2:10pm,Heritage_Bank_Stadium,0.0,18,2023,GCS,STK,0.0,0.0,0.0,...,18.0,6.0,4.0,277.0,380.0,4.0,8.0,349.0,396.0,5.0
4208,Sat 15 Jul 4:35pm,Marvel_Stadium,0.0,18,2023,CAR,POR,0.0,0.0,0.0,...,18.0,2.0,12.0,382.0,244.0,4.0,16.0,401.0,304.0,4.0
4209,Sat 15 Jul 7:10pm,Adelaide_Oval,0.0,18,2023,ADE,GWS,0.0,0.0,0.0,...,18.0,10.0,8.0,489.0,321.0,4.0,16.0,341.0,228.0,4.0
4210,Sat 15 Jul 7:25pm,GMHBA_Stadium,0.0,18,2023,GEE,ESS,0.0,0.0,0.0,...,18.0,5.0,10.0,329.0,290.0,4.0,8.0,336.0,320.0,4.0
4211,Sun 16 Jul 1:10pm,Marvel_Stadium,0.0,18,2023,NTH,HAW,0.0,0.0,0.0,...,18.0,16.0,0.0,294.0,471.0,4.0,4.0,256.0,371.0,4.0
4212,Sun 16 Jul 2:40pm,Optus_Stadium,0.0,18,2023,WCE,RCH,0.0,0.0,0.0,...,18.0,12.0,0.0,198.0,580.0,4.0,12.0,316.0,349.0,4.0


In [31]:
def prep_data(dataframe):
    
    dataframe = dataframe.drop(columns=['Date','Crowd'])
    dataframe.columns = dataframe.columns.str.replace(' ', '_')
    X = dataframe[['Venue', 'round', 'year', 'Home', 'Away',
        'pf_home', 'pa_home', 'per_home',
        'position_home', 'pf_away', 'pa_away',
        'per_away', 'position_away',
       'result_home_l5', 'pf_home_l5', 'pa_home_l5',
       'result_away_l5', 'pf_away_l5', 'pa_away_l5']]
    y = dataframe['target'].astype(int)
    
    X_encoded = pd.get_dummies(X, columns=['Venue','round','year','Home','Away'])
    
    return X_encoded, y

In [32]:
X_encoded, y = prep_data(df)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=1)

In [34]:
current_encoded, y_current = prep_data(current)

In [35]:
current_encoded = current_encoded.reindex(columns=X_encoded.columns, fill_value=0)

In [39]:
clf_xgb = xgb.XGBRegressor(objective='reg:squarederror', seed=1)

In [42]:
clf_xgb.fit(X_train,
           y_train,
           verbose=True,
           eval_metric='aucpr')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=1, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Optimise Parameters using Cross Validation and GridSearch()

In [69]:
param_grid = {
    'max_depth':[6,7,8,9],
    'learning_rate':[.1,.5,1],
    'gamma':[0,0.25,0.5],
    'reg_lambda':[0,1,10],
    'scale_pos_weight':[1,3,5]
}


In [70]:
optimal_params = GridSearchCV(
    estimator = xgb.XGBRegressor(objective='reg:squarederror',
                                  seed = 1,
                                  subsample=0.9,
                                  colsample_bytree=0.5),
    param_grid=param_grid,
    scoring='roc_auc',
    n_jobs=10,
    cv=3
)

In [71]:
optimal_params.fit(X_train,
                  y_train,
                   verbose=False
                  )


/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from panda

/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is de

/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.In

/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.In

/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} f

...

/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.In

/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.In

/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.In

/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/caseyh/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters w

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.5,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_...,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    seed=1, subsample=0.9, tree_meth

In [72]:
optimal_params.best_params_

{'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 6,
 'reg_lambda': 0,
 'scale_pos_weight': 1}

In [74]:
optimised = optimal_params.best_params_

clf_xgb = xgb.XGBRegressor(
           gamma = optimised['gamma'],
            learn_rate=optimised['learning_rate'],
            max_depth=optimised['max_depth'],
            reg_lambda=optimised['reg_lambda'],
            scale_pos_weight=optimised['scale_pos_weight'],
           subsample=0.9,
            colsample_bytree=0.5
           )
clf_xgb.fit(X_train,
              y_train,
               verbose=True
)

[02:07:05] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:576: 
Parameters: { "learn_rate" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learn_rate=0.1,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
             subsample=0.9, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [75]:
pd.Series(clf_xgb.feature_importances_, X_encoded.columns).sort_values(ascending=False).head(20)

Away_GCS               0.033647
Venue_GMHBA_Stadium    0.022130
position_away          0.019830
Away_FRE               0.015852
year_2019              0.015024
round_15               0.014955
Home_MEL               0.014187
year_2014              0.014165
Away_COL               0.014095
position_home          0.013605
year_2002              0.012704
Venue_UTAS_Stadium     0.012596
per_home               0.012404
Away_HAW               0.012361
Venue_TIO_Stadium      0.012132
year_2018              0.012092
round_20               0.011913
Home_CAR               0.011593
round_21               0.011576
Home_ADE               0.011492
dtype: float32

In [123]:
clf_xgb = xgb.XGBClassifier(
           eval_metric='aucpr',
           gamma = optimised['gamma'],
            learn_rate=optimised['learning_rate'],
            max_depth=optimised['max_depth'],
            reg_lambda=optimised['reg_lambda'],
            scale_pos_weight=optimised['scale_pos_weight'],
           subsample=0.9,
            colsample_bytree=0.5
            n_estimators=1
           )
clf_xgb.fit(X_train,y_train)

[00:02:47] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:576: 
Parameters: { "learn_rate" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, eval_metric='aucpr', gamma=0.25,
              gpu_id=-1, importance_type=None, interaction_constraints='',
              learn_rate=0.1, learning_rate=0.300000012, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=10, scale_pos_weight=5, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [124]:
bst = clf_xgb.get_booster()

for importance_type in ('weight','gain','cover','total_gain','total_cover'):
    print('%s: ' % importance_type, bst.get_score(importance_type=importance_type))
    
node_params= {'shape':'box','style':'filled, rounded','fillcolor':'#78cbe'}
    
leaf_params= {'shape':'box','style':'filled','fillcolor':'#e48038'}

xgb.to_graphviz(clf_xgb, num_trees=0, size="10,10",
               condition_node_params=node_params,
               leaf_node_params=leaf_params)

weight:  {'pf_home': 2.0, 's_round_home': 2.0, 'position_home': 1.0, 'per_away': 3.0, 'result_home_l5': 1.0, 'pf_away_l5': 2.0, 'pa_away_l5': 1.0, 'Venue_Blundstone Arena': 1.0, 'year_2014': 1.0}
gain:  {'pf_home': 9.102725982666016, 's_round_home': 10.358895301818848, 'position_home': 105.52734375, 'per_away': 27.025625228881836, 'result_home_l5': 6.712921142578125, 'pf_away_l5': 10.079957962036133, 'pa_away_l5': 5.078369140625, 'Venue_Blundstone Arena': 4.39288330078125, 'year_2014': 10.202972412109375}
cover:  {'pf_home': 60.875, 's_round_home': 102.875, 'position_home': 2199.25, 'per_away': 1009.0, 'result_home_l5': 81.0, 'pf_away_l5': 413.875, 'pa_away_l5': 960.25, 'Venue_Blundstone Arena': 205.5, 'year_2014': 210.75}
total_gain:  {'pf_home': 18.20545196533203, 's_round_home': 20.717790603637695, 'position_home': 105.52734375, 'per_away': 81.07687377929688, 'result_home_l5': 6.712921142578125, 'pf_away_l5': 20.159915924072266, 'pa_away_l5': 5.078369140625, 'Venue_Blundstone Arena'

ImportError: You must install graphviz to plot tree

In [76]:
preds = clf_xgb.predict(X_test)

In [77]:
X_test['prediction'] = preds

In [78]:
data=data.drop(columns='prediction_y')

KeyError: "['prediction_y'] not found in axis"

In [79]:
data = data.merge(X_test, left_index=True, right_index=True)

In [80]:
test = data[['Date','Venue','Crowd','round','year','Home','Away','Home Score','Away Score','target','prediction']]

In [81]:
test[test['Home Score']==test['Away Score']]

,Date,Venue,Crowd,round,year,Home,Away,Home Score,Away Score,target,prediction
589,Sun 8 Jun,Domain_Stadium,38540.0,11,2003,WCE,WBD,124.0,124.0,0.0,45.987457
1170,Sat 26 Aug 12:00am,GMHBA_Stadium,23068.0,21,2006,GEE,MEL,94.0,94.0,0.0,2.619007
1312,Fri 3 Aug 7:40pm,Marvel_Stadium,33600.0,18,2007,WBD,STK,85.0,85.0,0.0,3.193645
1659,Sat 8 Aug 7:10pm,MCG,41636.0,19,2009,ESS,BRL,87.0,87.0,0.0,36.677521
1775,Mon 14 Jun 2:10pm,MCG,67454.0,12,2010,MEL,COL,76.0,76.0,0.0,13.613328
1808,Fri 23 Jul 7:40pm,Marvel_Stadium,49373.0,17,2010,STK,HAW,87.0,87.0,0.0,5.258761
1856,Fri 1 Apr 7:40pm,MCG,41465.0,2,2011,STK,RCH,95.0,95.0,0.0,31.711926
1874,Sat 16 Apr 2:10pm,MCG,78065.0,4,2011,CAR,ESS,79.0,79.0,0.0,-18.737970
2284,Sat 18 May 7:40pm,SCG,22546.0,8,2013,SYD,FRE,70.0,70.0,0.0,31.753241


In [82]:
test[test['prediction']!=test['target']]

,Date,Venue,Crowd,round,year,Home,Away,Home Score,Away Score,target,prediction
11,Sat 18 Mar,AAMI_Stadium,24695.0,2,2000,POR,FRE,87.0,125.0,-38.0,13.852618
12,Sun 19 Mar,Princes_Park,21161.0,2,2000,CAR,HAW,154.0,112.0,42.0,30.251801
14,Sun 19 Mar,Marvel_Stadium,24460.0,2,2000,WBD,BRL,110.0,139.0,-29.0,12.876045
15,Sun 19 Mar,Domain_Stadium,38127.0,2,2000,WCE,SYD,70.0,82.0,-12.0,4.308372
19,Sat 25 Mar,MCG,20374.0,3,2000,HAW,BRL,110.0,103.0,7.0,14.473652
...,...,...,...,...,...,...,...,...,...,...,...
4167,Fri 9 Jun 7:50pm,Marvel_Stadium,23110.0,13,2023,WBD,POR,85.0,107.0,-22.0,15.225925
4181,Fri 23 Jun 7:50pm,Marvel_Stadium,28985.0,15,2023,STK,BRL,56.0,84.0,-28.0,29.206806
4183,Sat 24 Jun 5:25pm,Optus_Stadium,43063.0,15,2023,FRE,ESS,93.0,61.0,32.0,-0.416087
4194,Sun 2 Jul 2:50pm,TIO_Traegar_Park,3413.0,16,2023,MEL,GWS,45.0,47.0,-2.0,37.861130


In [83]:
preds = clf_xgb.predict(current_encoded)
current['target'] = preds

In [84]:
current

,Date,Venue,Crowd,round,year,Home,Away,Home Score,Away Score,target,...,s_round_away,position_away,result_home_l5,pf_home_l5,pa_home_l5,game played_home_l5,result_away_l5,pf_away_l5,pa_away_l5,game played_away_l5
4204,Thu 13 Jul 7:20pm,SCG,0.0,18,2023,SYD,WBD,0.0,0.0,13.799922,...,18.0,7.0,6.0,481.0,353.0,5.0,8.0,369.0,353.0,4.0
4205,Fri 14 Jul 7:50pm,MCG,0.0,18,2023,MEL,BRL,0.0,0.0,27.085377,...,18.0,3.0,8.0,253.0,245.0,4.0,16.0,504.0,323.0,5.0
4206,Sat 15 Jul 1:45pm,MCG,0.0,18,2023,COL,FRE,0.0,0.0,34.323105,...,18.0,14.0,12.0,353.0,265.0,4.0,4.0,317.0,452.0,5.0
4207,Sat 15 Jul 2:10pm,Heritage_Bank_Stadium,0.0,18,2023,GCS,STK,0.0,0.0,-12.513806,...,18.0,6.0,4.0,277.0,380.0,4.0,8.0,349.0,396.0,5.0
4208,Sat 15 Jul 4:35pm,Marvel_Stadium,0.0,18,2023,CAR,POR,0.0,0.0,-5.823423,...,18.0,2.0,12.0,382.0,244.0,4.0,16.0,401.0,304.0,4.0
4209,Sat 15 Jul 7:10pm,Adelaide_Oval,0.0,18,2023,ADE,GWS,0.0,0.0,13.632714,...,18.0,10.0,8.0,489.0,321.0,4.0,16.0,341.0,228.0,4.0
4210,Sat 15 Jul 7:25pm,GMHBA_Stadium,0.0,18,2023,GEE,ESS,0.0,0.0,26.407347,...,18.0,5.0,10.0,329.0,290.0,4.0,8.0,336.0,320.0,4.0
4211,Sun 16 Jul 1:10pm,Marvel_Stadium,0.0,18,2023,NTH,HAW,0.0,0.0,-10.453019,...,18.0,16.0,0.0,294.0,471.0,4.0,4.0,256.0,371.0,4.0
4212,Sun 16 Jul 2:40pm,Optus_Stadium,0.0,18,2023,WCE,RCH,0.0,0.0,-41.554794,...,18.0,12.0,0.0,198.0,580.0,4.0,12.0,316.0,349.0,4.0


In [85]:
import pickle

In [86]:
os.getcwd()

'/Users/caseyh/Desktop/footyfootyfooty/data_wrangling'

In [87]:
filename = 'xgboost_regression.sav'
path = '/Users/caseyh/Desktop/footyfootyfooty/machine_learning/models/'
pickle.dump(clf_xgb, open(path + filename, 'wb'))

In [ ]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(path + filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)